# 02_Tables

Some data for the tables is already plotted in [01_Figures](01_Figures.ipynb) next to the plots.
This analysis focuses on the latency per operation.

In [1]:
%run '00_Init.ipynb'

## Latency per Operation (Table 4)

### Dense Layers

In [2]:
filename = 'data/F-combined_no-cmsis_N_01ec_DenseBenchmark_varying-input_U-440_layer_results_2021-03-31.pkl'
df = pd.read_pickle(filename)
try:
    df = df.drop((df[df['error'].notna()]).index)
except:
    pass

df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

for i in range(0,500,1):
    string = f"_U-{i:03}"
    filter = df['model'].str.contains(string)
    df.loc[filter, 'dense_units'] = int(i)
    

filter = (df['layer'].str.contains('2_FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2 * (10*10) * df['dense_units']
df.loc[filter, 'layer_MACCs'] = df['dense_units'] * (10*10 + 1)

# of interest -> varying input
filter = (df['layer'].str.contains('3_FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2 * df['dense_units'] * 450
df.loc[filter, 'layer_MACCs'] = 450 * (df['dense_units']  + 1)

filter = (df['layer'].str.contains('4_FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2 * 450 * df['dense_units'] 
df.loc[filter, 'layer_MACCs'] = df['dense_units'] * (450  + 1)

# of interest -> varying input
filter = (df['layer'].str.contains('5_FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2* df['dense_units'] * 450
df.loc[filter, 'layer_MACCs'] = 450 * (df['dense_units']  + 1)

filter = (df['layer'].str.contains('6_FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2 * 450 * 10
df.loc[filter, 'layer_MACCs'] = 10 * (450  + 1)


filter = ~(df['layer'].str.contains('FULLY'))
df.loc[filter, 'layer_FLOPs'] = 100
df.loc[filter, 'layer_MACCs'] = 100


models = df['model'].unique()

for model in models:
    filter = (df['model'] == model)
    df_filtered = df.loc[filter]
    #display(df_filtered)
    df.loc[filter, 'layer_FLOPs_sum'] = df_filtered['layer_FLOPs'].sum()
    df.loc[filter, 'layer_FLOPs_relative'] = df_filtered['layer_FLOPs'] / df.loc[filter,'layer_FLOPs_sum']

In [3]:
df['MACCss'] =  (df['layer_latency_mean']) / df['layer_MACCs'] * 1e3

df['dense_type'] = np.where(df['dense_units'] % 2, 'odd', 'even')

filter = ~(df['dense_units'] % 4).astype(bool)
df.loc[filter, 'modulu'] = 'modulo 4'

cmsiss = ['none', 'cmsis-nn']
for cmsis in cmsiss:

    print(f"CMSIS-NN:{cmsis}")

    util_table = pd.DataFrame(columns=['MACCss_mean', 'MACss_std'])

    filter = (df['layer'].str.contains('3_FULLY')) | (df['layer'].str.contains('5_FULLY'))
    filter &= (df['cmsis-nn'] == cmsis)
    df_filtered = df.loc[filter]

    util_table.loc['all'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

    filter = (df['dense_type'] == 'odd')
    util_table.loc['odd'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())


    filter = (df['dense_type'] == 'even')
    util_table.loc['even'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

    filter = (df['modulu'] == 'modulo 4')
    util_table.loc['modulo 4'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())


    if cmsis == 'none':
        normalization = util_table.loc['all', 'MACCss_mean']

    util_table.loc[:,'factor'] = normalization / util_table['MACCss_mean']

    display(util_table)
    print("-----")

CMSIS-NN:none


,MACCss_mean,MACss_std,factor
all,145.18116691,0.06458177,1.00000000
odd,145.17513931,0.07290823,1.00004152
even,145.18719451,0.05480812,0.99995848
modulo 4,145.18965471,0.05426586,0.99994154


-----
CMSIS-NN:cmsis-nn


,MACCss_mean,MACss_std,factor
all,48.32929407,3.92498351,3.00399933
odd,50.26051150,3.30075981,2.88857321
even,46.39807664,3.54844435,3.12903416
modulo 4,44.78214276,2.97400116,3.24194328


-----


---

### Conv Layers

In [4]:
filename = 'data/F-combined_no-cmsis_N_01dg_ConvBenchmark_SingleConv_F-100_K-1_layer_results_2021-04-01.pkl'

df = pd.read_pickle(filename)
try:
    df = df.drop((df[df['error'].notna()]).index)
except:
    pass

df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

for i in range(1,5):
    string = f"_K-{i}_"
    filter = df['model'].str.contains(string)
    df.loc[filter, 'kernel'] = i
for i in range(1,200):
    string = f"_F-{i}_"
    filter = df['model'].str.contains(string)
    df.loc[filter, 'filters'] = i
    
# first layer has only a single input channel
filter = (df['layer'].str.contains('1_CONV'))
df.loc[filter, 'layer_FLOPs'] = 32*32 * 1 *  (2*df['kernel'] * df['kernel'] + 1) * df['filters']
df.loc[filter, 'layer_MACCs'] = 32*32 * 1 *  (df['kernel'] * df['kernel'] + 1) * df['filters']
    
filter = (df['layer'].str.contains('CONV') & ~(df['layer'].str.contains('1_CONV')))
df.loc[filter, 'layer_FLOPs'] = 32*32 * df['filters'] *  (2*df['kernel'] * df['kernel'] + 1) * 60
df.loc[filter, 'layer_MACCs'] = 32*32 * df['filters'] *  (df['kernel'] * df['kernel'] + 1) * 60

filter = (df['layer'].str.contains('FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2* (32*32 * 60) * 10
df.loc[filter, 'layer_MACCs'] = (32*32 * 60 + 1) * 10

filter = ~(df['layer'].str.contains('FULLY')) & ~(df['layer'].str.contains('CONV'))
df.loc[filter, 'layer_FLOPs'] = 10
df.loc[filter, 'layer_MACCs'] = 10



models = df['model'].unique()

for model in models:
    filter = (df['model'] == model)
    df_filtered = df.loc[filter]
    
    df.loc[filter, 'layer_FLOPs_sum'] = df_filtered['layer_FLOPs'].sum()
    df.loc[filter, 'layer_FLOPs_relative'] = df_filtered['layer_FLOPs'] / df.loc[filter,'layer_FLOPs_sum']
    
df.loc[:,'filters_kernel'] = 'F-' + df.loc[:,'filters'].astype(str) + '_K-' + df.loc[:,'kernel'].astype(str)


The table displays the results for cmsis-nn enabled and disabled, and 1x1 and 3x3 kernels.

In [5]:
df['MACCss'] =  (df['layer_latency_mean']) / df['layer_MACCs'] * 1e3

df['filter_type'] = np.where(df['filters'] % 2, 'odd', 'even')

filter = ~(df['filters'] % 4).astype(bool)
df.loc[filter, 'modulo'] = 'modulo 4'


cmsiss = ['none', 'cmsis-nn']
kernels = [1, 3]

for kernel in kernels:
    for cmsis in cmsiss:

        util_table = pd.DataFrame(columns=['MACCss_mean','MACCss_std'])

        print(f"CMSIS-NN:{cmsis}")
        print(f"Kernel = {kernel}")

    
        filter = (df['kernel'] == kernel) & (df['layer'].str.contains('2_CONV'))
        filter &= (df['cmsis-nn'] == cmsis)
        df_filtered = df.loc[filter]

        util_table.loc['all'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

        filter = (df['filter_type'] == 'odd')
        util_table.loc['odd'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())


        filter = (df['filter_type'] == 'even')
        util_table.loc['even'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

        filter = (df['modulo'] == 'modulo 4')
        util_table.loc['modulo 4'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

                
        if cmsis == 'none':
            normalization = util_table.loc['all', 'MACCss_mean']
            
        util_table.loc[:,'factor'] = normalization  / util_table['MACCss_mean'] 
        display(util_table)

        print("-------------")

CMSIS-NN:none
Kernel = 1


,MACCss_mean,MACCss_std,factor
all,148.51591278,0.43695807,1.00000000
odd,148.51612929,0.43656512,0.99999854
even,148.51571595,0.45862232,1.00000133
modulo 4,148.50611824,0.50559180,1.00006595


-------------
CMSIS-NN:cmsis-nn
Kernel = 1


,MACCss_mean,MACCss_std,factor
all,24.66270360,1.55508679,6.02188289
odd,25.98111598,0.26810614,5.71630229
even,23.46414689,1.20560932,6.32948274
modulo 4,22.51244153,0.67101148,6.59705934


-------------
CMSIS-NN:none
Kernel = 3


,MACCss_mean,MACCss_std,factor
all,250.60842612,0.26760817,1.00000000
odd,250.61023786,0.26055764,0.99999277
even,250.60677908,0.28656716,1.00000657
modulo 4,250.60911639,0.32368767,0.99999725


-------------
CMSIS-NN:cmsis-nn
Kernel = 3


,MACCss_mean,MACCss_std,factor
all,37.44382660,1.55504929,6.69291707
odd,39.00583504,0.06041837,6.42489581
even,36.02381892,0.41849357,6.95674233
modulo 4,35.66395729,0.07086640,7.02693826


-------------


---

### Depthwise Conv

In [6]:
filename = 'data/F-combined_no-cmsis_N_01f_Depthwise-Conv_f-100_K-1_layer_results_2021-04-01.pkl'

df = pd.read_pickle(filename)
try:
    df = df.drop((df[df['error'].notna()]).index)
except:
    pass

df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

for i in range(1,5):
    string = f"_K-{i}_"
    filter = df['model'].str.contains(string)
    df.loc[filter, 'kernel'] = i
for i in range(1,200):
    string = f"_f-{i}_"
    filter = df['model'].str.contains(string)
    df.loc[filter, 'filters'] = i
    
# first layer has only a single input channel
filter = (df['layer'].str.contains('1_CONV'))
df.loc[filter, 'layer_FLOPs'] = 24*24 * 1 *  (2*df['kernel'] * df['kernel'] + 1) * df['filters']
df.loc[filter, 'layer_MACCs'] = 24*24 * 1 *  (df['kernel'] * df['kernel'] + 1) * df['filters']
    
filter = (df['layer'].str.contains('2_DEPTHWISE_CONV_2D'))
# 
df.loc[filter, 'layer_FLOPs'] = 24*24 * df['filters'] *  (2*df['kernel'] * df['kernel'] + 1) 
df.loc[filter, 'layer_MACCs'] = 24*24 * df['filters'] *  (df['kernel'] * df['kernel'] + 1)


filter = (df['layer'].str.contains('FULLY'))
df.loc[filter, 'layer_FLOPs'] = 2* (24*24 * df['filters']) * 10
df.loc[filter, 'layer_MACCs'] = (24*24* df['filters'] + 1) * 10

filter = ~(df['layer'].str.contains('FULLY')) & ~(df['layer'].str.contains('CONV'))
df.loc[filter, 'layer_FLOPs'] = 10
df.loc[filter, 'layer_MACCs'] = 10



models = df['model'].unique()

for model in models:
    filter = (df['model'] == model)
    df_filtered = df.loc[filter]
    
    df.loc[filter, 'layer_FLOPs_sum'] = df_filtered['layer_FLOPs'].sum()
    df.loc[filter, 'layer_FLOPs_relative'] = df_filtered['layer_FLOPs'] / df.loc[filter,'layer_FLOPs_sum']
    
df.loc[:,'filters_kernel'] = 'F-' + df.loc[:,'filters'].astype(str) + '_K-' + df.loc[:,'kernel'].astype(str)


In [7]:
df['MACCss'] =  (df['layer_latency_mean']) / df['layer_MACCs'] * 1e3

df['filter_type'] = np.where(df['filters'] % 2, 'odd', 'even')

filter = ~(df['filters'] % 4).astype(bool)
df.loc[filter, 'modulo'] = 'modulo 4'

cmsiss = ['none', 'cmsis-nn']
kernels = [1, 3]
for kernel in kernels:
    for cmsis in cmsiss:
    
        print(f"CMSIS-NN:{cmsis}")
        print(f"Kernel = {kernel}")

        util_table = pd.DataFrame(columns=['MACCss_mean', 'MACCss_std'])


        filter = (df['kernel'] == kernel) & (df['layer'].str.contains('2_DEPTHWISE_CONV_2D'))
        filter &= (df['cmsis-nn'] == cmsis)
        df_filtered = df.loc[filter]

        util_table.loc['all'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

        filter = (df['filter_type'] == 'odd')
        util_table.loc['odd'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())


        filter = (df['filter_type'] == 'even')
        util_table.loc['even'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

        filter = (df['modulo'] == 'modulo 4')
        util_table.loc['modulo 4'] = (df_filtered.loc[filter, 'MACCss'].mean(), df_filtered.loc[filter, 'MACCss'].std())

        
        if cmsis == 'none':
            normalization = util_table.loc['all', 'MACCss_mean']
            
        util_table.loc[:,'factor'] =normalization / util_table['MACCss_mean'] 

        
        display(util_table)
        print('----------')


CMSIS-NN:none
Kernel = 1


,MACCss_mean,MACCss_std,factor
all,925.05282883,1.19662555,1.00000000
odd,924.99462214,1.00938820,1.00006293
even,925.10574400,1.39297498,0.99994280
modulo 4,924.71075937,1.69990475,1.00036992


----------
CMSIS-NN:cmsis-nn
Kernel = 1


,MACCss_mean,MACCss_std,factor
all,581.82899580,4.00105695,1.58990500
odd,584.15136011,2.20886009,1.58358414
even,579.71775553,4.16284900,1.59569518
modulo 4,576.59747009,2.43579464,1.60433036


----------
CMSIS-NN:none
Kernel = 3


,MACCss_mean,MACCss_std,factor
all,602.80668381,0.33400540,1.00000000
odd,602.76855857,0.41721800,1.00006325
even,602.84134312,0.25234998,0.99994251
modulo 4,602.73152878,0.22083718,1.00012469


----------
CMSIS-NN:cmsis-nn
Kernel = 3


,MACCss_mean,MACCss_std,factor
all,270.49207773,8.26380232,2.22855578
odd,278.36435738,0.90243408,2.16553114
even,263.33545986,4.18762759,2.28912082
modulo 4,259.77012631,0.92591067,2.32053890


----------


#### 